In [1]:
import re
import pandas as pd
from itertools import product

def results_text_to_clipboard_df(text: str):
    """
    Parse lines like:
      [train[canada_wildfires_2016_lb5_set1]] macro-F1 = 0.29588704978190755
    Build a DF with columns: event, 5_1, 5_2, 5_3, 10_1, ... 50_3
    Copy DF to clipboard for easy paste into Sheets.
    """
    # event, lb size, set number, score
    rx = re.compile(
        r"\[train\[(.+?)_lb(\d+)_set(\d+)\]\]\s*macro-F1\s*=\s*([0-9]*\.?[0-9]+)",
        flags=re.IGNORECASE
    )

    rows = []
    for m in rx.finditer(text):
        event, lb, setnum, score = m.group(1), m.group(2), m.group(3), m.group(4)
        rows.append((event, int(lb), int(setnum), float(score)))

    if not rows:
        print("No matches found. Check the text format or regex.")
        return None

    # Build wide table
    df_long = pd.DataFrame(rows, columns=["event", "lb", "set", "score"])
    # Keep only the requested lb/set grid
    wanted_lbs  = [5, 10, 25, 50]
    wanted_sets = [1, 2, 3]
    df_long = df_long[df_long["lb"].isin(wanted_lbs) & df_long["set"].isin(wanted_sets)]

    # Pivot to event rows, columns like "5_1"
    df_wide = df_long.assign(col=lambda d: d["lb"].astype(str) + "_" + d["set"].astype(str)) \
                     .pivot_table(index="event", columns="col", values="score", aggfunc="first")

    # Ensure full cartesian column order exists
    all_cols = [f"{lb}_{s}" for lb, s in product(wanted_lbs, wanted_sets)]
    df_wide = df_wide.reindex(columns=all_cols)

    # Put 'event' as an actual column (since you asked for it in the output)
    df_out = df_wide.reset_index()

    # Copy to clipboard
    # df_out.to_clipboard(index=False)
    print("✅ Copied to clipboard. Preview:")
    print(df_out)
    return df_out

# ---- Example usage ----
text = """[train[hurricane_irma_2017_lb5_set1]] macro-F1 = 0.4241789252765972
[train[hurricane_irma_2017_lb5_set2]] macro-F1 = 0.1973404828074534
[train[hurricane_irma_2017_lb5_set3]] macro-F1 = 0.20155903238276068
[train[kerala_floods_2018_lb5_set1]] macro-F1 = 0.20287086659206116
[train[kerala_floods_2018_lb5_set2]] macro-F1 = 0.2311844711852332
[train[kerala_floods_2018_lb5_set3]] macro-F1 = 0.2585996888881726
[train[california_wildfires_2018_lb5_set1]] macro-F1 = 0.39315958447551524
[train[california_wildfires_2018_lb5_set2]] macro-F1 = 0.2858703349970175
[train[california_wildfires_2018_lb5_set3]] macro-F1 = 0.31952181697654997
[train[hurricane_florence_2018_lb5_set1]] macro-F1 = 0.40551075149132343
[train[hurricane_florence_2018_lb5_set2]] macro-F1 = 0.2212394920062789
[train[hurricane_florence_2018_lb5_set3]] macro-F1 = 0.2853856806833145
[train[hurricane_harvey_2017_lb5_set1]] macro-F1 = 0.2601605523815206
[train[hurricane_harvey_2017_lb5_set2]] macro-F1 = 0.47651302602835366
[train[hurricane_harvey_2017_lb5_set3]] macro-F1 = 0.4232666521077427
[train[kaikoura_earthquake_2016_lb5_set1]] macro-F1 = 0.4462589080072576
[train[kaikoura_earthquake_2016_lb5_set2]] macro-F1 = 0.431948744312611
[train[kaikoura_earthquake_2016_lb5_set3]] macro-F1 = 0.4166028198133061
[train[hurricane_maria_2017_lb5_set1]] macro-F1 = 0.3614200773982207
[train[hurricane_maria_2017_lb5_set2]] macro-F1 = 0.5410043610752613
[train[hurricane_maria_2017_lb5_set3]] macro-F1 = 0.2557951184486098
[train[canada_wildfires_2016_lb5_set1]] macro-F1 = 0.3925216862924965
[train[canada_wildfires_2016_lb5_set2]] macro-F1 = 0.3665705873422772
[train[canada_wildfires_2016_lb5_set3]] macro-F1 = 0.36455364841638205
[train[cyclone_idai_2019_lb5_set1]] macro-F1 = 0.3392898913376615
[train[cyclone_idai_2019_lb5_set2]] macro-F1 = 0.311025037548443
[train[cyclone_idai_2019_lb5_set3]] macro-F1 = 0.3417957981425921
[train[hurricane_dorian_2019_lb5_set1]] macro-F1 = 0.2412311148468409
[train[hurricane_dorian_2019_lb5_set2]] macro-F1 = 0.2667363164558088
[train[hurricane_dorian_2019_lb5_set3]] macro-F1 = 0.3870779781286657
[train[hurricane_irma_2017_lb10_set1]] macro-F1 = 0.3593180957434044
[train[hurricane_irma_2017_lb10_set2]] macro-F1 = 0.42238662185734527
[train[hurricane_irma_2017_lb10_set3]] macro-F1 = 0.4483240954556533
[train[kerala_floods_2018_lb10_set1]] macro-F1 = 0.3905538934727239
[train[kerala_floods_2018_lb10_set2]] macro-F1 = 0.4409706219370244
[train[kerala_floods_2018_lb10_set3]] macro-F1 = 0.3915433172777571
[train[california_wildfires_2018_lb10_set1]] macro-F1 = 0.5736884184341307
[train[california_wildfires_2018_lb10_set2]] macro-F1 = 0.4850751294744356
[train[california_wildfires_2018_lb10_set3]] macro-F1 = 0.3783010763681359
[train[hurricane_florence_2018_lb10_set1]] macro-F1 = 0.4703271817616229
[train[hurricane_florence_2018_lb10_set2]] macro-F1 = 0.4432360748174436
[train[hurricane_florence_2018_lb10_set3]] macro-F1 = 0.5016821066151325
[train[hurricane_harvey_2017_lb10_set1]] macro-F1 = 0.5574435343156315
[train[hurricane_harvey_2017_lb10_set2]] macro-F1 = 0.45348380007984723
[train[hurricane_harvey_2017_lb10_set3]] macro-F1 = 0.4710612369093524
[train[kaikoura_earthquake_2016_lb10_set1]] macro-F1 = 0.624729808886237
[train[kaikoura_earthquake_2016_lb10_set2]] macro-F1 = 0.5751158442399182
[train[kaikoura_earthquake_2016_lb10_set3]] macro-F1 = 0.5749641766372845
[train[hurricane_maria_2017_lb10_set1]] macro-F1 = 0.504664028839788
[train[hurricane_maria_2017_lb10_set2]] macro-F1 = 0.5734986380972963
[train[hurricane_maria_2017_lb10_set3]] macro-F1 = 0.5411018222160938
[train[canada_wildfires_2016_lb10_set1]] macro-F1 = 0.4856706849328148
[train[canada_wildfires_2016_lb10_set2]] macro-F1 = 0.5052991956070709
[train[canada_wildfires_2016_lb10_set3]] macro-F1 = 0.4230096290628763
[train[cyclone_idai_2019_lb10_set1]] macro-F1 = 0.416505963767832
[train[cyclone_idai_2019_lb10_set2]] macro-F1 = 0.4493053541468914
[train[cyclone_idai_2019_lb10_set3]] macro-F1 = 0.46562658606819074
[train[hurricane_dorian_2019_lb10_set1]] macro-F1 = 0.45855266900702163
[train[hurricane_dorian_2019_lb10_set2]] macro-F1 = 0.45460472449271794
[train[hurricane_dorian_2019_lb10_set3]] macro-F1 = 0.5441273416465146
[train[hurricane_irma_2017_lb25_set1]] macro-F1 = 0.5934396538934232
[train[hurricane_irma_2017_lb25_set2]] macro-F1 = 0.5671484618807437
[train[hurricane_irma_2017_lb25_set3]] macro-F1 = 0.5803252573103994
[train[kerala_floods_2018_lb25_set1]] macro-F1 = 0.5107984836351247
[train[kerala_floods_2018_lb25_set2]] macro-F1 = 0.5185056581751755
[train[kerala_floods_2018_lb25_set3]] macro-F1 = 0.43139832922839577
[train[california_wildfires_2018_lb25_set1]] macro-F1 = 0.5950496452069866
[train[california_wildfires_2018_lb25_set2]] macro-F1 = 0.5901323229630027
[train[california_wildfires_2018_lb25_set3]] macro-F1 = 0.6073106991518358
[train[hurricane_florence_2018_lb25_set1]] macro-F1 = 0.6535290229726592
[train[hurricane_florence_2018_lb25_set2]] macro-F1 = 0.6455214374440835
[train[hurricane_florence_2018_lb25_set3]] macro-F1 = 0.667403521399838
[train[hurricane_harvey_2017_lb25_set1]] macro-F1 = 0.5891058354283178
[train[hurricane_harvey_2017_lb25_set2]] macro-F1 = 0.5631500423077553
[train[hurricane_harvey_2017_lb25_set3]] macro-F1 = 0.5723655373577101
[train[kaikoura_earthquake_2016_lb25_set1]] macro-F1 = 0.6843962952938899
[train[kaikoura_earthquake_2016_lb25_set2]] macro-F1 = 0.6305057184251673
[train[kaikoura_earthquake_2016_lb25_set3]] macro-F1 = 0.7205749548501124
[train[hurricane_maria_2017_lb25_set1]] macro-F1 = 0.6027187354799053
[train[hurricane_maria_2017_lb25_set2]] macro-F1 = 0.6302360779768601
[train[hurricane_maria_2017_lb25_set3]] macro-F1 = 0.6233004077630029
[train[canada_wildfires_2016_lb25_set1]] macro-F1 = 0.5517302668297205
[train[canada_wildfires_2016_lb25_set2]] macro-F1 = 0.5120078372968048
[train[canada_wildfires_2016_lb25_set3]] macro-F1 = 0.5964496305768684
[train[cyclone_idai_2019_lb25_set1]] macro-F1 = 0.4919140825378068
[train[cyclone_idai_2019_lb25_set2]] macro-F1 = 0.6085595757900576
[train[cyclone_idai_2019_lb25_set3]] macro-F1 = 0.5846270101472042
[train[hurricane_dorian_2019_lb25_set1]] macro-F1 = 0.534121987288797
[train[hurricane_dorian_2019_lb25_set2]] macro-F1 = 0.5844062434696361
[train[hurricane_dorian_2019_lb25_set3]] macro-F1 = 0.5835011554781353
[train[hurricane_irma_2017_lb50_set1]] macro-F1 = 0.6113122750660388
[train[hurricane_irma_2017_lb50_set2]] macro-F1 = 0.6241924209941855
[train[hurricane_irma_2017_lb50_set3]] macro-F1 = 0.6448034641105805
[train[kerala_floods_2018_lb50_set1]] macro-F1 = 0.5119283491830704
[train[kerala_floods_2018_lb50_set2]] macro-F1 = 0.5699098611108588
[train[kerala_floods_2018_lb50_set3]] macro-F1 = 0.5388711013565608
[train[california_wildfires_2018_lb50_set1]] macro-F1 = 0.6304478470267411
[train[california_wildfires_2018_lb50_set2]] macro-F1 = 0.6552178301450517
[train[california_wildfires_2018_lb50_set3]] macro-F1 = 0.6041081856569813
[train[hurricane_florence_2018_lb50_set1]] macro-F1 = 0.6931170580178244
[train[hurricane_florence_2018_lb50_set2]] macro-F1 = 0.6811267755939989
[train[hurricane_florence_2018_lb50_set3]] macro-F1 = 0.6946629359251676
[train[hurricane_harvey_2017_lb50_set1]] macro-F1 = 0.6420852459680169
[train[hurricane_harvey_2017_lb50_set2]] macro-F1 = 0.6547725384967227
[train[hurricane_harvey_2017_lb50_set3]] macro-F1 = 0.6613459814736755
[train[kaikoura_earthquake_2016_lb50_set1]] macro-F1 = 0.6619523787588207
[train[kaikoura_earthquake_2016_lb50_set2]] macro-F1 = 0.6425404148401861
[train[kaikoura_earthquake_2016_lb50_set3]] macro-F1 = 0.7036043104824375
[train[hurricane_maria_2017_lb50_set1]] macro-F1 = 0.6483791223089701
[train[hurricane_maria_2017_lb50_set2]] macro-F1 = 0.6419930480093198
[train[hurricane_maria_2017_lb50_set3]] macro-F1 = 0.6510603062064275
[train[canada_wildfires_2016_lb50_set1]] macro-F1 = 0.5451736346822461
[train[canada_wildfires_2016_lb50_set2]] macro-F1 = 0.5088933525912597
[train[canada_wildfires_2016_lb50_set3]] macro-F1 = 0.5193673028244158
[train[cyclone_idai_2019_lb50_set1]] macro-F1 = 0.6351695114554212
[train[cyclone_idai_2019_lb50_set2]] macro-F1 = 0.6415550062842249
[train[cyclone_idai_2019_lb50_set3]] macro-F1 = 0.5902674955345735
[train[hurricane_dorian_2019_lb50_set1]] macro-F1 = 0.5604834556456993
[train[hurricane_dorian_2019_lb50_set2]] macro-F1 = 0.5959336041174017
[train[hurricane_dorian_2019_lb50_set3]] macro-F1 = 0.5519865923435995"""
results_text_to_clipboard_df(text)


✅ Copied to clipboard. Preview:
col                      event       5_1       5_2       5_3      10_1  \
0    california_wildfires_2018  0.393160  0.285870  0.319522  0.573688   
1        canada_wildfires_2016  0.392522  0.366571  0.364554  0.485671   
2            cyclone_idai_2019  0.339290  0.311025  0.341796  0.416506   
3        hurricane_dorian_2019  0.241231  0.266736  0.387078  0.458553   
4      hurricane_florence_2018  0.405511  0.221239  0.285386  0.470327   
5        hurricane_harvey_2017  0.260161  0.476513  0.423267  0.557444   
6          hurricane_irma_2017  0.424179  0.197340  0.201559  0.359318   
7         hurricane_maria_2017  0.361420  0.541004  0.255795  0.504664   
8     kaikoura_earthquake_2016  0.446259  0.431949  0.416603  0.624730   
9           kerala_floods_2018  0.202871  0.231184  0.258600  0.390554   

col      10_2      10_3      25_1      25_2      25_3      50_1      50_2  \
0    0.485075  0.378301  0.595050  0.590132  0.607311  0.630448  0.655218   

col,event,5_1,5_2,5_3,10_1,10_2,10_3,25_1,25_2,25_3,50_1,50_2,50_3
0,california_wildfires_2018,0.393160,0.285870,0.319522,0.573688,0.485075,0.378301,0.595050,0.590132,0.607311,0.630448,0.655218,0.604108
1,canada_wildfires_2016,0.392522,0.366571,0.364554,0.485671,0.505299,0.423010,0.551730,0.512008,0.596450,0.545174,0.508893,0.519367
2,cyclone_idai_2019,0.339290,0.311025,0.341796,0.416506,0.449305,0.465627,0.491914,0.608560,0.584627,0.635170,0.641555,0.590267
3,hurricane_dorian_2019,0.241231,0.266736,0.387078,0.458553,0.454605,0.544127,0.534122,0.584406,0.583501,0.560483,0.595934,0.551987
4,hurricane_florence_2018,0.405511,0.221239,0.285386,0.470327,0.443236,0.501682,0.653529,0.645521,0.667404,0.693117,0.681127,0.694663
5,hurricane_harvey_2017,0.260161,0.476513,0.423267,0.557444,0.453484,0.471061,0.589106,0.563150,0.572366,0.642085,0.654773,0.661346
6,hurricane_irma_2017,0.424179,0.197340,0.201559,0.359318,0.422387,0.448324,0.593440,0.567148,0.580325,0.611312,0.624192,0.644803
7,hurricane_maria_2017,0.361420,0.541004,0.255795,0.504664,0.573499,0.541102,0.602719,0.630236,0.623300,0.648379,0.641993,0.651060
8,kaikoura_earthquake_2016,0.446259,0.431949,0.416603,0.624730,0.575116,0.574964,0.684396,0.630506,0.720575,0.661952,0.642540,0.703604
9,kerala_floods_2018,0.202871,0.231184,0.258600,0.390554,0.440971,0.391543,0.510798,0.518506,0.431398,0.511928,0.569910,0.538871
